In [ ]:
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize


In [ ]:

filename = 'tag wiki-formatted.txt'
with open(filename,'r',encoding='UTF-8') as fp:
    lines = fp.readlines()

In [ ]:
result_1=[]
for line in lines:
    line = eval(line)
    result_1.append(line)
    

In [ ]:

tup_list = [tuple(item.items()) for item in result_1]
sets = {t for t in tup_list}
result_2 = [dict(t) for t in sets]

# print(result_2)

In [ ]:
#  extract all subtrees marked as NP
def extract_np_subtrees(text):
    tagged = pos_tag(word_tokenize(text))
    # A block analyzerbased on regular expressions
    chunk_parser = nltk.RegexpParser(r" NP:{<NN.*>}")
    # perform chunking analysis on already annotated text
    tree = chunk_parser.parse(tagged)
    return [subtree for subtree in tree.subtrees() if subtree.label() == 'NP']


In [ ]:
result_3=[]
for item in result_2:
    for k,v in item.items():
#         if len(v)<=1 or v.endswith('yet!'):
#             break
        temp={k:[]}
        np_subtrees = extract_np_subtrees(v)
        for subtree in np_subtrees:
            pos= subtree.leaves()
            temp[k].append(pos[0][0])
        result_3.append(temp)
        
        
        

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r'../recommend-output.csv')
formLibList = list(df['fromLib'])
toLibList = list(df['toLib'])
confidenceList = list(df['confidence'])
confirmedList = list(df['isConfirmed'])


In [ ]:
tagAll=[]
n = len(formLibList)
for i in range(n):
    item=[]
    item.append(formLibList[i])
    item.append(toLibList[i])
    item.append(confidenceList[i])
    item.append(confirmedList[i])
    tagAll.append(item)

In [ ]:
print(len(tagAll))
print(tagAll[:10])

In [ ]:
for item1 in tagAll:
    for item2 in result_3:
        lib1 = item1[0].split(':')[1]
        lib2 = list(item2.keys())[0]
        if lib1==lib2 :
            item1.insert(1,item2[lib2])
            break

In [ ]:
print(len(tagAll))
print(tagAll[:10])

In [ ]:
'''
Multiple operations are required to completely delete items with a length less than 5
'''
for item in tagAll:
    if len(item)<5:
        tagAll.remove(item)

In [ ]:
print(len(tagAll))

In [ ]:
for item1 in tagAll:
    for item2 in result_3:
        try:
            lib1 = item1[2].split(':')[1]
            lib2 = list(item2.keys())[0]
            if lib1==lib2 :
                item1.insert(3,item2[lib2])
                break
        except IndexError:
            break

In [ ]:
for item in tagAll:
    if len(item)<6:
        tagAll.remove(item)

In [ ]:
print(len(tagAll))

In [ ]:
print(tagAll[:10])

In [ ]:
def count_common_words(s1, s2):
    set1 = set(s1)
    set2 = set(s2)
    common_words = set1.intersection(set2)
    return len(common_words)

In [ ]:
for item in tagAll:
    num = count_common_words(item[1],item[3])
    if num==0:
        item.append(1)
    else:
        item.append(1.5 * num)

In [ ]:
print(len(tagAll))


In [ ]:
for item in tagAll:
    item[1] = ','.join(item[1])
    item[3] = ','.join(item[3])

In [ ]:
print(tagAll[:3])

In [ ]:
import pandas as pd
data = tagAll
df = pd.DataFrame(data, columns= 
                  ['fromLib', 'fromLabels', 'toLib', 'toLabels','confidence','isConfirmed','labelSupport'])
df.to_excel(r'../resources/tag wiki.xlsx', index=False)